In [1]:
import xspec as xs
import pandas as pd
import os
import numpy as np

##### Column density value don't freeze initially. If the column density estimates are one or more order less than the galactic value then freeze. Also check the significance of the parameters. Save the .xcm file. 

In [2]:
cwd = os.getcwd()
os.chdir(f'{cwd}')
pd.read_csv('spec_info.csv')

,source_no.,obsid,avg_galactic_nh,grouping,redshift
0,source1,9183,5.180000e+20,20,0.622590
1,source2,888,7.770000e+19,20,0.562437
2,source3,4936,6.900000e+19,20,1.205077
3,source5,4613,7.610000e+20,20,NaN
4,source5,794,7.610000e+20,20,NaN
...,...,...,...,...,...
220,source229,11688,4.650000e+20,20,NaN
221,source230,3347,5.530000e+19,20,0.720000
222,source231,12718,1.370000e+20,20,0.431876
223,source232,16169,4.870000e+20,20,1.635000


In [3]:
list_file = pd.read_csv('spec_info.csv')
source_no = list_file['source_no.']
obsids = list_file['obsid']
gal_nh = list_file['avg_galactic_nh'] # Absolute value with unit as 1
grp = list_file['grouping']
z = list_file['redshift']

gal_nh_22 = gal_nh*1e-22 # In units of 10^22


In [4]:
cwd

'/home/samrat/264_3sigma_sources_o.net_cnts_more_than_400'

In [5]:
list_file['source_no.'][39]

'source31'

In [6]:
gal_nh_22[40]

0.017

In [10]:
x = 'nan'
x == 'nan'

True

In [7]:
#Empty array where the result will be stored
nh = []
nh_hi_err = []
nh_lo_err = []

gamma = []
gamma_hi_err = []
gamma_lo_err = []

norm = []
norm_hi_err = []
norm_lo_err = []

flux = []
flux_hi_err = []
flux_lo_err = []

lumin = []
lumin_hi_err = []
lumin_lo_err = []

dof = []
stat = []

n = int(input('How many?'))

for i in range(n):
  
    spec_path = f'{source_no[i]}/{obsids[i]}/repro/spectrum/'
    os.chdir(f'{cwd}/{spec_path}')
#     xs.Xset.openLog('xspec.log')
    if grp[i] != 1 :
        xs.Fit.statMethod = "chi"
    else:
        xs.Fit.statMethod = "cstat"

    #Loading spectra and model
    s = xs.Spectrum('spec_grp.pi')
    m = xs.Model("tbabs*pow", setPars = {1:gal_nh_22[i], 2:2})
    xs.Plot.xAxis = 'keV'
    s.ignore('**-0.5 7.0-**')
    par1 = m.TBabs.nH
    par2 = m.powerlaw.PhoIndex
    par3 = m.powerlaw.norm
   
    
    flag = False
    
    try : 
        xs.Fit.perform()
        par1_value = par1.values[0]
        par2_value = par2.values[0]
        par3_value = par3.values[0]
    except :
#         raise "XSPEC Error:  No variable parameters for fit"
        flag = True
        
  
    #if estimated nh is less than one order of magnitude lower then freeze it
    if par1_value < gal_nh_22[i]/10 or par1_value > 10^2 or flag == True: 

#     if True : # Freezing nH for all  
        m.setPars(f'{gal_nh_22[i]}')
        par1.frozen = True
        xs.Fit.perform()
        par1_value = par1.values[0]
        par2_value = par2.values[0]
        par3_value = par3.values[0]
        
        try :
            xs.Fit.error("2-3")

            #collecting the values and error of the parameters 
            #(Remember these all are not the absolute value rather the values at the given units)
            par1_lo_err = 0
            par1_hi_err = 0

            par2_lo_err = par2_value - par2.error[0]
            par2_hi_err = par2.error[1] - par2_value

            
            par3_lo_err = par3_value - par3.error[0]
            par3_hi_err = par3.error[1] - par3_value
        except :
            par1_lo_err = 0
            par1_hi_err = 0
            par2_lo_err = 'nan'
            par2_hi_err = 'nan'
            par3_lo_err = 'nan'
            par3_hi_err = 'nan'
    else :
        #collecting the values and error of the parameters (Remember these all are not the absolute value rather 
        #the values at the given units)
        try :
            xs.Fit.error("1-3")
            par1_lo_err = par1_value - par1.error[0]
            par1_hi_err = par1.error[1] - par1_value

            par2_lo_err = par2_value - par2.error[0]
            par2_hi_err = par2.error[1] - par2_value

            par3_lo_err = par3_value - par3.error[0]
            par3_hi_err = par3.error[1] - par3_value
        
        except :
            par1_lo_err = 'nan'
            par1_hi_err = 'nan'
            par2_lo_err = 'nan'
            par2_hi_err = 'nan'
            par3_lo_err = 'nan'
            par3_hi_err = 'nan'

    xs.AllModels.calcFlux('0.5 7.0 error')
    flux_i = s.flux

    #store values for the different sources in the array
    nh.append(par1_value*1e+22)
    if np.is_nan()
    nh_hi_err.append(par1_hi_err*1e+22)
    nh_lo_err.append(par1_lo_err*1e+22)

    gamma.append(par2_value)
    gamma_hi_err.append(par2_hi_err)
    gamma_lo_err.append(par2_lo_err)

    norm.append(par3_value)
    norm_hi_err.append(par3_hi_err)
    norm_lo_err.append(par3_hi_err)

    flux.append(flux_i[0])
    flux_lo_err.append(flux_i[0] - flux_i[1])
    flux_hi_err.append(flux_i[2] - flux_i[0])

    stat.append(xs.Fit.statistic)
    dof.append(xs.Fit.dof)
    
    if np.invert(np.isnan(z[i])) :
        xs.AllModels.calcLumin(f'0.5 7.0 {z[i]} error')
        lumin_i = s.lumin
        
        lumin.append(lumin_i[0] * 1e+44)
        lumin_lo_err.append((lumin_i[0] - lumin_i[1])*1e+44)
        lumin_hi_err.append((lumin_i[2] - lumin_i[0])*1e+44)
    
    else :
        lumin.append('nan')
        lumin_lo_err.append('nan')
        lumin_hi_err.append('nan')
        
    
#     xs.Xset.save(fileName = 'tbabs*pow',info ='a')
#     xs.Xset.closeLog()
#     xs.Plot.device = "/xs"
#     xs.Plot.iplot('data','resid')
#     xs.Plot.show()
    xs.AllData.clear()
    os.chdir(f'{cwd}')


How many?225
Default fit statistic is set to: Chi-Squared
   This will apply to all current and newly loaded spectra.

1 spectrum  in use
 
Spectral Data File: spec_grp.pi  Spectrum 1
Net count rate (cts/s) for Spectrum:1  8.975e-02 +/- 1.577e-03 (99.3 % total)
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-127
  Telescope: CHANDRA Instrument: ACIS  Channel Type: PI
  Exposure Time: 3.644e+04 sec
 Using fit statistic: chi
 Using Background File                spec_bkg.pi
  Background Exposure Time: 3.644e+04 sec
 Using Response (RMF) File            spec.rmf for Source 1
 Using Auxiliary Response (ARF) File  spec.corr.arf


Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   powerlaw   PhoIndex            1.00000      +/-  0.0          
   3    2   powerlaw   norm                1.00000      +/-  0.0          
___________

tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.


  0.2743   0.9616   0.0001  
----------------------------------------

  Covariance Matrix
        1           2           3   
   3.467e-04   8.717e-04   1.106e-07
   8.717e-04   3.154e-03   3.330e-07
   1.106e-07   3.330e-07   4.273e-11
------------------------------------

Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    5.36493E-02  +/-  1.86192E-02  
   2    2   powerlaw   PhoIndex            1.35372      +/-  5.61587E-02  
   3    2   powerlaw   norm                1.12873E-04  +/-  6.53651E-06  
________________________________________________________________________


Fit statistic  : Chi-Squared                   95.99     using 118 bins.

Test statistic : Chi-Squared                   95.99     using 118 bins.
 Null hypothesis probability of 9.01e-01 with 115 degrees of freedom
 Parameter   Confidence Range (2.706)
     1    0.0238487    0.0848196    (-0.0298015,0.031169


1 spectrum  in use
 
Spectral Data File: spec_grp.pi  Spectrum 1
Net count rate (cts/s) for Spectrum:1  3.381e-02 +/- 6.124e-04 (98.3 % total)
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-121
  Telescope: CHANDRA Instrument: ACIS  Channel Type: PI
  Exposure Time: 9.224e+04 sec
 Using fit statistic: chi
 Using Background File                spec_bkg.pi
  Background Exposure Time: 9.224e+04 sec
 Using Response (RMF) File            spec.rmf for Source 1
 Using Auxiliary Response (ARF) File  spec.corr.arf


Fit statistic  : Chi-Squared                 2196.91     using 121 bins.

Test statistic : Chi-Squared                 2196.91     using 121 bins.
 Null hypothesis probability of 0.00e+00 with 119 degrees of freedom
 Current data and model not fit yet.

Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   powerlaw   Ph

Fit statistic  : Chi-Squared              8.527813e+11     using 45 bins.

Test statistic : Chi-Squared              8.527813e+11     using 45 bins.
 Null hypothesis probability of 0.000000e+00 with 42 degrees of freedom
 Current data and model not fit yet.

                                   Parameters
Chi-Squared  |beta|/N    Lvl          1:nH    2:PhoIndex        3:norm
46.5891      118.544      -3      0.106675       1.78250   3.39158e-05
46.2668      144839       -4      0.116117       1.81598   3.44319e-05
46.2658      443.465      -5      0.116881       1.81952   3.45252e-05
 Variances and Principal Axes
                 1        2        3  
 1.2673E-12| -0.0001  -0.0000   1.0000  
 2.6796E-04|  0.9707  -0.2401   0.0001  
 1.4873E-02|  0.2401   0.9707   0.0000  
----------------------------------------

  Covariance Matrix
        1           2           3   
   1.110e-03   3.404e-03   1.159e-07
   3.404e-03   1.403e-02   4.012e-07
   1.159e-07   4.012e-07   1.395e-11
---------

...

et.

Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    3.02000E-02  +/-  0.0          
   2    2   powerlaw   PhoIndex            2.00000      +/-  0.0          
   3    2   powerlaw   norm                1.00000      +/-  0.0          
________________________________________________________________________

     1 channels (1) ignored in spectrum #     1
     2 channels (22-23) ignored in spectrum #     1

Fit statistic  : Chi-Squared              3.881305e+10     using 20 bins.

Test statistic : Chi-Squared              3.881305e+10     using 20 bins.
 Null hypothesis probability of 0.000000e+00 with 17 degrees of freedom
 Current data and model not fit yet.

                                   Parameters
Chi-Squared  |beta|/N    Lvl          1:nH    2:PhoIndex        3:norm
19.2192      0.551309     -3     0.0230952       1.97334   9.73624e-05
19.2189      264.437      -4     0.


Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   powerlaw   PhoIndex            1.00000      +/-  0.0          
   3    2   powerlaw   norm                1.00000      +/-  0.0          
________________________________________________________________________


Fit statistic  : Chi-Squared              8.339119e+11     using 27 bins.

Test statistic : Chi-Squared              8.339119e+11     using 27 bins.
 Null hypothesis probability of 0.000000e+00 with 24 degrees of freedom
 Current data and model not fit yet.

Fit statistic  : Chi-Squared              3.011400e+11     using 27 bins.

Test statistic : Chi-Squared              3.011400e+11     using 27 bins.
 Null hypothesis probability of 0.000000e+00 with 24 degrees of freedom
 Current data and model not fit yet.

Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Com

***Warning: Parameter pegged at hard limit: 0



1 spectrum  in use
 
Spectral Data File: spec_grp.pi  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.748e-02 +/- 7.782e-04 (97.9 % total)
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-26
  Telescope: CHANDRA Instrument: ACIS  Channel Type: PI
  Exposure Time: 2.973e+04 sec
 Using fit statistic: chi
 Using Background File                spec_bkg.pi
  Background Exposure Time: 2.973e+04 sec
 Using Response (RMF) File            spec.rmf for Source 1
 Using Auxiliary Response (ARF) File  spec.corr.arf


Fit statistic  : Chi-Squared                 6008.30     using 26 bins.

Test statistic : Chi-Squared                 6008.30     using 26 bins.
 Null hypothesis probability of 0.00e+00 with 23 degrees of freedom
 Current data and model not fit yet.

Model TBabs<1>*powerlaw<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   powerlaw   PhoInd

TypeError: can't multiply sequence by non-int of type 'float'

In [8]:
i

143

In [ ]:
os.chdir(f'{cwd}/')
spec_output = open('spectral_modelling_result_0.5-7kev_test_thaw_nh_for_high_value.csv','a')
#uncomment the following line only if there is no such file and that is for the first time only
spec_output.write('''source_no.,obid,flux,flux_hi_err,flux_lo_err,redshift,lumin,lumin_hi_err,lumin_lo_err,nh,nh_hi_err,nh_lo_err,gamma,gamma_hi_err,gamma_low_err,norm,norm_hi_err,norm_lo_err,stat,dof,red_stat\n''')

#load the output of the modelling to the csv file
for i in range(n):
    print(i)
    spec_output.write(f'{source_no[i]},{obsids[i]},{flux[i]},{flux_hi_err[i]},{flux_lo_err[i]},{z[i]},{lumin[i]},{lumin_hi_err[i]},{lumin_lo_err[i]},{nh[i]},{nh_hi_err[i]},{nh_lo_err[i]},{gamma[i]},{gamma_hi_err[i]},{gamma_lo_err[i]},{norm[i]},{norm_hi_err[i]},{norm_lo_err[i]},{stat[i]},{dof[i]},{stat[i]/dof[i]}\n')

spec_output.close()